In [ ]:
pip install pygam

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from pygam import LinearGAM, GAM, s, f, l

In [ ]:
#Load the data into a pandas dataframe
wage=pd.read_csv("../input/wagedata/Wage.csv")
wage['education']=wage['education'].replace(["1. < HS Grad","2. HS Grad","4. College Grad","3. Some College","5. Advanced Degree"], ["1","2","4","3","5"])
wage['education']=wage['education'].astype(str).astype(int)
wage.head()
#wage.shape

In [ ]:
#Split the data into test and train sets 
train, test = train_test_split(wage, test_size=0.25)
X_train = train[['year','age','education']]
y_train = train['wage']
X_test = test[['year','age','education']]
y_test = test['wage']

In [ ]:
#GAM 
gam=GAM(s(0,n_splines=6)+s(1,n_splines=6)+s(2,n_splines=5)).fit(X_train,y_train)
XX = gam.generate_X_grid
plt.rcParams['figure.figsize'] = (28, 8)
fig, axs = plt.subplots(1, 3)
titles = list(X_train.columns)
for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
    ax.set_title(titles[i]);
plt.show()

In [ ]:
gam_preds = gam.predict(X_test)
gam_preds2=gam.predict(X_train)

In [ ]:
plt.title('Actual values vs Predictions')
x=range(750)
plt.rcParams['figure.figsize'] = (10,10)
plt.scatter(x,y_test)
plt.scatter(x,gam_preds)
plt.ylabel("Wage")
plt.legend(["Actual values", "GAM Predictions"])

plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
plt.title("Residual Distribution")
plt.hist(y_test-gam_preds, bins=20, rwidth=0.9,color='#607c8e')
plt.show()

In [ ]:
r2_test=r2_score(y_test,gam_preds)
r2_train=r2_score(y_train,gam_preds2)

print(r2_test)
print(r2_train)